In [1]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import random
import numpy
import pandas as pd
import os
import multiprocessing
import time

import matplotlib.pyplot as plt
import seaborn as sns

import Knapsack

# Genetic Algorithm constants:
POPULATION_SIZE = 50
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.1   # probability for mutating an individual
MAX_GENERATIONS = 200
HALL_OF_FAME_SIZE = 1


# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)


# fitness calculation
def knapsackValue(individual):
    return knapsack_info.getValue(individual),  # return a tuple

def SolveKnapsack(file_path):
    # create the knapsack problem instance to be used:
    global knapsack_info
    knapsack_info = Knapsack.Knapsack01Problem(file_path)

    toolbox = base.Toolbox()

    # create an operator that randomly returns 0 or 1:
    toolbox.register("zeroOrOne", random.randint, 0, 1)

    # define a single objective, maximizing fitness strategy:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))

    # create the Individual class based on list:
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create the individual operator to fill up an Individual instance:
    toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, len(knapsack_info))

    # create the population operator to generate a list of individuals:
    toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

    toolbox.register("evaluate", knapsackValue)

    # genetic operators:mutFlipBit

    # Tournament selection with tournament size of 3:
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Single-point crossover:
    toolbox.register("mate", tools.cxTwoPoint)

    # Flip-bit mutation:
    # indpb: Independent probability for each attribute to be flipped
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/len(knapsack_info))

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
    
    
#     # prepare the statistics object:
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("max", numpy.max)
#     stats.register("avg", numpy.mean)
    
#     start_time = time.process_time()  # start time
#     processor_solve_knapsack = multiprocessing.Process(target = algorithms.eaSimple,
#                                  args = (population, toolbox, P_CROSSOVER, P_MUTATION, MAX_GENERATIONS, stats, hof, True))
#     print(1)
#     processor_solve_knapsack.start()
#     processor_solve_knapsack.join()
    
#     end_time = time.process_time()    # end time

#     if processor_solve_knapsack.is_alive():
#         print("running... let's kill it...")

#         # Terminate - may not work if process is stuck for good
#         processor_solve_knapsack.terminate()
#         # OR Kill - will work for sure, no chance for process to finish nicely however
#         # p.kill()

#         processor_solve_knapsack.join()
    
    start_time = time.process_time()  # start time
    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=MAX_GENERATIONS, halloffame=hof, verbose=False)

    end_time = time.process_time()    # end time



    # print best solution found:
    best = hof.items[0]
    print("-- Best Ever Fitness = ", best.fitness.values[0])

    del creator.FitnessMax, creator.Individual, knapsack_info, toolbox
    
    return int(best.fitness.values[0]), (end_time - start_time)


# Genetic Algorithm flow:

test_plan_paths = "C:\\Users\\Hieu\\Desktop\\AI\\Bai_taps\\KnapSack\\kplib-master\\"

output = {'STT':[], 'File_path':[], 'Computed_value':[], 'Time':[]}

excel_file = pd.read_excel("test_result.xlsx")
number_of_test_file = 1
for file_info_path in excel_file["File_path"]:
    full_path = os.path.join(test_plan_paths, file_info_path + ".kp")

    if os.path.isfile(full_path):
        print("stt: ", number_of_test_file)
        computed_value, computed_time = SolveKnapsack(full_path)

        output['STT'].append(number_of_test_file)
        output['File_path'].append(file_info_path)
        output['Computed_value'].append(computed_value)
        output['Time'].append(computed_time)  

        number_of_test_file += 1

df_result = pd.DataFrame.from_dict(output)
print(df_result)
df_result.to_excel("output_GA.xlsx", index = False)


stt:  1
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\00Uncorrelated\n01000\R01000\s065.kp
-- Best Ever Fitness =  324853.0
stt:  2
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\00Uncorrelated\n00050\R10000\s019.kp
-- Best Ever Fitness =  180190.0
stt:  3
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\00Uncorrelated\n05000\R01000\s037.kp
-- Best Ever Fitness =  1371607.0
stt:  4
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\00Uncorrelated\n10000\R10000\s032.kp
-- Best Ever Fitness =  26229362.0
stt:  5
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\00Uncorrelated\n00200\R01000\s043.kp
-- Best Ever Fitness =  76011.0
stt:  6
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\01WeaklyCorrelated\n00500\R01000\s094.kp
-- Best Ever Fitness =  136062.0
stt:  7
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\01WeaklyCorrelated\n00050\R01000\s025.kp
-- Best Ever Fitness =  12256.0
stt:  8
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kp

-- Best Ever Fitness =  234297.0
stt:  59
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\11ProfitCeiling\n00050\R01000\s040.kp
-- Best Ever Fitness =  12204.0
stt:  60
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\11ProfitCeiling\n00500\R10000\s090.kp
-- Best Ever Fitness =  1229586.0
stt:  61
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\12Circle\n10000\R10000\s056.kp
-- Best Ever Fitness =  1658046335.0
stt:  62
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\12Circle\n02000\R10000\s062.kp
-- Best Ever Fitness =  332772402.0
stt:  63
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\12Circle\n01000\R10000\s028.kp
-- Best Ever Fitness =  165994058.0
stt:  64
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\12Circle\n00200\R01000\s085.kp
-- Best Ever Fitness =  1028445.0
stt:  65
C:\Users\Hieu\Desktop\AI\Bai_taps\KnapSack\kplib-master\12Circle\n05000\R01000\s035.kp
-- Best Ever Fitness =  25881363.0
    STT                          File